## Downloading Dependencies

In [ ]:
 ! pip install -q kaggle

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.1MB 2.7MB/s 
     |████████████████████████████████| 3.0MB 16.9MB/s 
     |████████████████████████████████| 890kB 23.5MB/s 
     |████████████████████████████████| 1.1MB 42.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=b08d3b4b30dbacb80e41df98a630f3ec50e524034c0de88a9838e985990e6a6a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle 

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

## Connecting with Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive 
!ls /mydrive

/mydrive


In [ ]:
ls

drive/  sample_data/


In [ ]:
pwd

'/content'

## Path for data download

In [ ]:
cd /content/drive/My Drive/sentiment analysis/input

/content/drive/My Drive/sentiment analysis/input


In [ ]:
!kaggle datasets download -d abhishek/bert-base-uncased

 97% 378M/389M [00:06<00:00, 54.1MB/s]
100% 389M/389M [00:06<00:00, 63.9MB/s]


In [ ]:
pwd

'/content/drive/My Drive/sentiment analysis/input'

In [ ]:
!ls

 bert_base_unchased  'IMDB Dataset.csv'


In [ ]:
! unzip bert-base-uncased.zip

Archive:  bert-base-uncased.zip
  inflating: config.json             
  inflating: pytorch_model.bin       
  inflating: vocab.txt               


In [ ]:
!rm -r bert-base-uncased.zip

In [ ]:
!mv config.json pytorch_model.bin vocab.txt bert_base_unchased/

## Training

Change path to working directory


In [ ]:
cd /content/drive/My Drive/sentiment analysis/src

/content/drive/My Drive/sentiment analysis/src


In [ ]:
import logging
logging.basicConfig(level=logging.ERROR)

In [ ]:
!python train.py

2020-10-13 16:08:43.867970: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length

## Prediction

In [ ]:
%cd /content/drive/My Drive/sentiment analysis/src

/content/drive/My Drive/sentiment analysis/src


In [ ]:
import config
import torch
import time
from model import BERTBaseUncased
import functools
import torch.nn as nn
import datetime

In [ ]:
MODEL = None
DEVICE = config.DEVICE
PREDICTION_DICT = dict()


def sentence_prediction(sentence):
    tokenizer = config.TOKENIZER
    max_len = config.MAX_LEN
    review = str(sentence)
    review = " ".join(review.split())

    inputs = tokenizer.encode_plus(
        review, None, add_special_tokens=True, max_length=max_len
    )

    ids = inputs["input_ids"]
    mask = inputs["attention_mask"]
    token_type_ids = inputs["token_type_ids"]

    padding_length = max_len - len(ids)
    ids = ids + ([0] * padding_length)
    mask = mask + ([0] * padding_length)
    token_type_ids = token_type_ids + ([0] * padding_length)

    ids = torch.tensor(ids, dtype=torch.long).unsqueeze(0)
    mask = torch.tensor(mask, dtype=torch.long).unsqueeze(0)
    token_type_ids = torch.tensor(token_type_ids, dtype=torch.long).unsqueeze(0)

    ids = ids.to(DEVICE, dtype=torch.long)
    token_type_ids = token_type_ids.to(DEVICE, dtype=torch.long)
    mask = mask.to(DEVICE, dtype=torch.long)

    outputs = MODEL(ids=ids, mask=mask, token_type_ids=token_type_ids)

    outputs = torch.sigmoid(outputs).cpu().detach().numpy()
    return outputs[0][0]


def predict(sentence):
    sentence = sentence
    start_time = time.time()
    positive_prediction = sentence_prediction(sentence)
    negative_prediction = 1 - positive_prediction
    response = {}
    response["response"] = {
        "time": str(datetime.datetime.now())[:-7],
        "positive": str(positive_prediction),
        "negative": str(negative_prediction),
        "sentence": str(sentence),
        "time_taken": str(time.time() - start_time),
    }
    return response



In [ ]:
MODEL = BERTBaseUncased()
MODEL.load_state_dict(torch.load(config.MODEL_PATH))
MODEL.to(DEVICE)
MODEL.eval()

## Enter Prediction text in predict 
response = predict("Simple Online and Realtime Tracking (SORT) is a pragmatic approach to multiple object tracking with a focus on simple effective algorithms")

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
response["response"]["time"]

'2020-10-15 11:30:30.'

In [ ]:
response["response"]["positive"]

'0.98250556'

In [ ]:
response["response"]["negative"]

'0.01749444007873535'

In [ ]:
response["response"]["sentence"]

'Simple Online and Realtime Tracking (SORT) is a pragmatic approach to multiple object tracking with a focus on simple effective algorithms'

In [ ]:
response["response"]["time_taken"]

'0.03959536552429199'

In [ ]:
response['response']

{'negative': '0.01749444007873535',
 'positive': '0.98250556',
 'sentence': 'Simple Online and Realtime Tracking (SORT) is a pragmatic approach to multiple object tracking with a focus on simple effective algorithms',
 'time': '2020-10-15 11:30:46',
 'time_taken': '0.03959536552429199'}